In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## imports

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


# 1) MNIST Dataset & Dataloaders


In [ ]:
transform = transforms.ToTensor()
train_data = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_data  = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_data, batch_size=64, shuffle=False)

100%|██████████| 9.91M/9.91M [00:00<00:00, 137MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 49.5MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 70.5MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 10.4MB/s]


# 3) Simple CNN


In [ ]:


class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, 3)   # 1→16 filters, 3x3
        self.pool  = nn.MaxPool2d(2, 2)
        self.fc    = nn.Linear(16 * 13 * 13, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))   # conv → relu → pool
        x = x.view(x.size(0), -1)              # flatten
        return self.fc(x)

# 4) Training Function

In [ ]:
def train_model(model, loader, epochs=1):
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    loss_fn = nn.CrossEntropyLoss()
    for _ in range(epochs):
        for imgs, labels in loader:
            preds = model(imgs)
            loss = loss_fn(preds, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


# 5) Test Accuracy

In [ ]:
def test_accuracy(model):
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in test_loader:
            preds = model(imgs)
            correct += (preds.argmax(1) == labels).sum().item()
            total += labels.size(0)
    return correct / total

# ---- RUN EXPERIMENT ---- #

In [ ]:


fc = 
cnn = SimpleCNN()

print("Training FC Model...")
train_model(fc, train_loader, epochs=1)  # 1 epoch for fast demo
print("Training CNN Model...")
train_model(cnn, train_loader, epochs=1)

fc_acc = test_accuracy(fc)
cnn_acc = test_accuracy(cnn)

print("\n=== RESULTS ===")
print(f"Fully Connected Accuracy: {fc_acc*100:.2f}%")
print(f"CNN Accuracy:            {cnn_acc*100:.2f}%")
print("\nCNN performs better because it learns SPATIAL features (edges, shapes) from the image instead of treating it as flat pixels.")


Training FC Model...
Training CNN Model...

=== RESULTS ===
Fully Connected Accuracy: 86.38%
CNN Accuracy:            90.41%

CNN performs better because it learns SPATIAL features (edges, shapes) from the image instead of treating it as flat pixels.
